In [93]:
# formatting to splink
import os
import pandas as pd
import altair as alt
alt.renderers.enable('html')
# import data
import numpy as np

data_set = 2
df = pd.read_csv(f"corrupted\\scen3\\20231027_data{data_set}.csv") 

# formatting
# drop useless rows?
column_names = df.columns.tolist()
columns_to_keep = ['unique_id' , 'maternal_agecat', 'ethgroup', 'g1_gender_arc', 'imddecile', 'g1_dob_arc1', 'syn_g0_surname', 'syn_g1_surname', 'syn_g1_firstname'] 
df = df.loc[:, columns_to_keep]
df["g1_dob_arc1"] = pd.to_datetime(df["g1_dob_arc1"], unit = "D")

df["g1_dob_str"] = df["g1_dob_arc1"].astype(str)
df["g1_dob_str"] = df["g1_dob_str"].str[:10]
df["g1_dob_arc1"] = df["g1_dob_str"]
df.drop(columns = "g1_dob_str")

# inspect vars, code "missing" into nulls.
df['maternal_agecat'] = df['maternal_agecat'].replace('missing', np.nan)
df['ethgroup'] = df['ethgroup'].replace('missing', np.nan)
df['imddecile'] = df['imddecile'].replace('missing', np.nan)
df['syn_g1_firstname'] = df['syn_g1_firstname'].replace('missing', np.nan)
df['syn_g0_surname'] = df['syn_g0_surname'].replace('missing', np.nan)
df['syn_g1_surname'] = df['syn_g1_surname'].replace('missing', np.nan)

columns_to_keep = ['unique_id' , 'maternal_agecat', 'ethgroup', 'g1_gender_arc', 'imddecile', 'g1_dob_arc1', 'syn_g0_surname', 'syn_g1_surname', 'syn_g1_firstname'] 
df = df.loc[:, columns_to_keep]
df.head(5)


# import uncorrupted data
directory = os.path.join(os.getcwd(), "output\\independent")
file_name = f"data_{data_set}_independent.csv"
file_path = os.path.join(directory, file_name)
undrawn_df = pd.read_csv(file_path)

# create gold standard (with cluster), by keeping uncorrupted = True rows.
undrawn_df['uncorrupted_record'] = undrawn_df['uncorrupted_record'].astype(bool)
gold_df = undrawn_df[undrawn_df['uncorrupted_record']]
columns_to_keep = ['unique_id' , 'maternal_agecat', 'ethgroup', 'g1_gender_arc', 'imddecile', 'g1_dob_arc1', 'syn_g0_surname', 'syn_g1_surname', 'syn_g1_firstname'] 
gold_df = gold_df.loc[:, columns_to_keep]
gold_df["g1_dob_arc1"] = pd.to_datetime(gold_df["g1_dob_arc1"], unit = "D")
#gold_df['g1_dob_arc1']=pd.to_datetime(gold_df['g1_dob_arc1'], format = '%d/%m/%Y') 

gold_df["g1_dob_str"] = gold_df["g1_dob_arc1"].astype(str)
gold_df["g1_dob_str"] = gold_df["g1_dob_str"].str[:10]
gold_df["g1_dob_arc1"] = gold_df["g1_dob_str"]
gold_df.drop(columns = "g1_dob_str")

gold_df['maternal_agecat'] = gold_df['maternal_agecat'].replace('', np.nan)
gold_df['ethgroup'] = gold_df['ethgroup'].replace('Missing', np.nan)
gold_df['imddecile'] = gold_df['imddecile'].replace('', np.nan)
gold_df['syn_g1_firstname'] = gold_df['syn_g1_firstname'].replace('missing', np.nan)
gold_df['syn_g0_surname'] = gold_df['syn_g0_surname'].replace('missing', np.nan)
gold_df['syn_g1_surname'] = gold_df['syn_g1_surname'].replace('missing', np.nan)

columns_to_keep = ['unique_id' , 'maternal_agecat', 'ethgroup', 'g1_gender_arc', 'imddecile', 'g1_dob_arc1', 'syn_g0_surname', 'syn_g1_surname', 'syn_g1_firstname'] 
gold_df = gold_df.loc[:, columns_to_keep]

gold_df.head(5)
# gold_df['g1_dob_arc1']=pd.to_datetime(df['g1_dob_arc1'].astype(str), format='%Y-%m-%d')



,unique_id,maternal_agecat,ethgroup,g1_gender_arc,imddecile,g1_dob_arc1,syn_g0_surname,syn_g1_surname,syn_g1_firstname
0,90000001,30-39,Asian,Female,NaN,1992-03-30,khan,khan,Pearl
1001,90000002,20-29,Asian,Female,4.0,1992-11-11,fitzgerald,fitzgerald,Liyana
2002,90000003,30-39,Asian,Female,2.0,1992-11-05,tang,tang,Aishah
3003,90000004,20-29,Asian,Female,3.0,1991-07-09,chen,chen,Lina
4004,90000005,30-39,Asian,Female,2.0,1991-10-22,begum,begum,Hannah


In [2]:
from splink.duckdb.linker import DuckDBLinker
linker = DuckDBLinker(df)
linker.profile_columns()



alt.VConcatChart(...)

In [6]:
linker.missingness_chart()

alt.LayerChart(...)

In [7]:
linker = DuckDBLinker(gold_df)
linker.profile_columns()

alt.VConcatChart(...)

In [9]:
linker.missingness_chart()

alt.LayerChart(...)

In [3]:
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

# link_only - setting 1: basic comparison
settings_deterministic =  {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        block_on(["syn_g1_firstname", "syn_g1_surname", "syn_g0_surname", "g1_dob_arc1"]),
        block_on(["syn_g1_firstname", "syn_g0_surname", "g1_dob_arc1", "g1_gender_arc"]),
        block_on(["syn_g1_surname", "syn_g0_surname", "g1_dob_arc1", "g1_gender_arc"]),
        block_on(["syn_g1_firstname", "syn_g1_surname", "g1_dob_arc1", "g1_gender_arc"]),
        block_on(["syn_g1_firstname", "syn_g1_surname", "syn_g0_surname", "g1_gender_arc"]),
    ],  
    "comparisons": [],
    "retain_matching_columns":True,
    "retain_intermediate_calculation_columns":True,
    "additional_columns_to_retain": [
        "ethgroup",
        "imddecile",
        "maternal_agecat",
    ]
}

linker = DuckDBLinker(
    [gold_df, df],
    settings_dict= settings_deterministic,
    input_table_aliases=["df_left","df_right"]
)

# linker.save_model_to_json(out_path = "linkage_outputs\\scenario1_dataset1\\deterministic_linkage_settings.json", overwrite = True)
linker.cumulative_num_comparisons_from_blocking_rules_chart()

alt.Chart(...)

In [4]:
df_predict = linker.deterministic_link()
df_predict.as_pandas_dataframe().head()
deterministic_scen1 = df_predict.as_pandas_dataframe()
deterministic_scen1.to_csv("linkage_outputs\\scen0\\dataset5\\deterministic.csv")

In [5]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=1)


Completed iteration 1, root rows count 65
Completed iteration 2, root rows count 2
Completed iteration 3, root rows count 0


In [7]:
clusters.as_pandas_dataframe(limit=5)
linker.cluster_studio_dashboard(df_predict, clusters, out_path = "linkage_outputs\\scenario1_dataset1\\dashboard.html", sampling_method = 'by_cluster_size', overwrite = True )

from IPython.display import IFrame

IFrame(
    src="./linkage_outputs/scenario1_dataset1/dashboard.html", width = "100%", height = 1200
)


In [94]:
# Define Comparisons - lower JW weights should not matter, try anyway. remove date 3 months.
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_level_library as cll

comparison_syn_g1_first_name = {
    "output_column_name": "syn_g1_firstname",
    "comparison_description": "G1 First name jaro winkler",
    "comparison_levels": [
        {
            "sql_condition": "syn_g1_firstname_l IS NULL OR syn_g1_firstname_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "syn_g1_firstname_l = syn_g1_firstname_r",
            "label_for_charts": "Exact match",
        },
        {
            "sql_condition": 'jaro_winkler_similarity("syn_g1_firstname_l", "syn_g1_firstname_r") >= 0.8',
            "label_for_charts": "Jaro_winkler_similarity 0.8",

        },
        {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
    ],
}

comparison_syn_g0_surname = {
    "output_column_name": "syn_g0_surname",
    "comparison_description": "G0 surname jaro winkler",
    "comparison_levels": [
        {
            "sql_condition": "syn_g0_surname_l IS NULL OR syn_g0_surname_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "syn_g0_surname_l = syn_g0_surname_r",
            "label_for_charts": "Exact match",
        },
        {
            "sql_condition": 'jaro_winkler_similarity("syn_g0_surname_l", "syn_g0_surname_r") >= 0.8',
            "label_for_charts": "Jaro_winkler_similarity 0.8",

        },
        {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
    ],
}

comparison_syn_g1_surname = {
    "output_column_name": "syn_g1_surname",
    "comparison_description": "G1 surname jaro winkler",
    "comparison_levels": [
        {
            "sql_condition": "syn_g1_surname_l IS NULL OR syn_g1_surname_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "syn_g1_surname_l = syn_g1_surname_r",
            "label_for_charts": "Exact match",
        },
        {
            "sql_condition": 'jaro_winkler_similarity("syn_g1_surname_l", "syn_g1_surname_r") >= 0.8',
            "label_for_charts": "Jaro_winkler_similarity 0.8",

        },
        {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
    ],
}
"""
dob_comparison = {
            "output_column_name": "g1_dob_arc1",
            "comparison_description": "Exact match vs. Date_Of_Birth within levenshtein thresholds 1, 2 vs. Dates within the following thresholds Year(s): 1, Year(s): 10 vs. anything else",
            "comparison_levels": [
                cll.null_level("g1_dob_arc1"),
                cll.exact_match_level("g1_dob_arc1"),
                cll.levenshtein_level("g1_dob_arc1", 1),
                cll.levenshtein_level("g1_dob_arc1", 2),
                cll.datediff_level("g1_dob_arc1",
                                    date_threshold=1,
                                    date_metric="year"),
                cll.datediff_level("g1_dob_arc1",
                                    date_threshold=10,
                                    date_metric="year"),
                cll.else_level(),
            ],
        }

comparison_date = {
    "output_column_name": "g1_dob_arc1",
    "comparison_description": "exact match vs dob within 3 months vs anything else",
    "comparison_levels": [
        cll.null_level("g1_dob_arc1"),
        cll.exact_match_level("g1_dob_arc1"),
        cll.else_level(),
    ],
}

"""
comparison_date = {
    "output_column_name": "g1_dob_arc1",
    "comparison_description": "exact match vs anything else",
    "comparison_levels": [
        cll.null_level("g1_dob_arc1"),
        cll.exact_match_level("g1_dob_arc1", m_probability =  0.999),
        cll.else_level(m_probability = 0.001),
    ],
}

comparison_gender = {
    "output_column_name": "g1_gender_arc",
    "comparison_description": "exact match vs anything else",
    "comparison_levels": [
        cll.null_level("g1_gender_arc"),
        cll.exact_match_level("g1_gender_arc", m_probability =  0.999),
        cll.else_level(m_probability = 0.001),
    ],
}


In [61]:
# Scenario 2, relaxed blocking rules, strategy 2
from splink.duckdb.blocking_rule_library import block_on
setting_test = {
        "link_type": "link_only",
    "probability_two_random_records_match": 1/13281,
    "blocking_rules_to_generate_predictions": [       
        block_on(["syn_g1_firstname"]),
        block_on(["syn_g1_surname"]),
        block_on(["syn_g0_surname"]),
        block_on(["g1_dob_arc1", "g1_gender_arc"]),
    ]
}

linker = DuckDBLinker(
    [gold_df, df],
    settings_dict= setting_test,
    input_table_aliases=["df_left","df_right"]
)


linker.cumulative_num_comparisons_from_blocking_rules_chart()

alt.Chart(...)

In [95]:
# Scenario 2, simple probabilistic linkage, blocking strategy 3
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl


settings_prob = {
    "link_type": "link_only",
    "probability_two_random_records_match": 1/13281,
    "blocking_rules_to_generate_predictions": [       
        block_on(["syn_g1_firstname"]),
        block_on(["syn_g1_surname"]),
        block_on(["syn_g0_surname"]),
        block_on(["g1_dob_arc1", "g1_gender_arc"]),
    ],  
    "comparisons":[
        comparison_syn_g1_first_name,
        comparison_syn_g1_surname,
        comparison_syn_g0_surname,
        comparison_date,
        comparison_gender,     
    ],
    "retain_matching_columns":True,
    "retain_intermediate_calculation_columns":True,
    "additional_columns_to_retain": [
        "ethgroup",
        "imddecile",
        "maternal_agecat",
    ]
}
# set retain to false to run quickly
#linker.save_model_to_json(out_path = "linkage_outputs\\scenario1_dataset1\\probablistic_linkage_settings.json", overwrite = True)

linker = DuckDBLinker(
    [gold_df, df],
    settings_dict= settings_prob,
    input_table_aliases=["df_left","df_right"]
)


In [96]:
# Estimate U using random sampling
linker.estimate_u_using_random_sampling(max_pairs=2e7, seed = 1)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - syn_g1_firstname (no m values are trained).
    - syn_g1_surname (no m values are trained).
    - syn_g0_surname (no m values are trained).


In [97]:
# Use labelled data for m probability.
linker.estimate_m_from_label_column("unique_id")

------- Estimating m probabilities using from column unique_id -------
m probability not trained for g1_dob_arc1 - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.
m probability not trained for g1_gender_arc - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.

Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [9]:
linker.match_weights_chart()


alt.VConcatChart(...)

In [10]:
linker.m_u_parameters_chart()

alt.HConcatChart(...)

In [14]:
linker.match_weights_histogram(linker.predict())
df = linker.predict(threshold_match_weight= -15)
records = df.as_record_dict(limit=20)
linker.waterfall_chart(records)
#linker.match_weights_histogram(df)

alt.LayerChart(...)

In [19]:
df_predictions = linker.predict()

linker.comparison_viewer_dashboard(df_predictions, "scv.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame
IFrame(
    src="scv.html", width="100%", height=1200
)


In [98]:
results = linker.predict()
results.as_pandas_dataframe(limit = 5)
prob_linkage = results.as_pandas_dataframe()
prob_linkage.to_csv(f"linkage_outputs\\scen0\\dataset{data_set}\\probabilistic_nothreshold.csv")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
# With threshold
threshold_values = [-0.22, 0.00000001, 2, 3, 4, 5, 6]
for v in threshold_values:
    results = linker.predict(threshold_match_weight = v)
    prob_linkage = results.as_pandas_dataframe()
    file_name = f"linkage_outputs//scen0//dataset5//probabilistic_threshold_{v}.csv"
    prob_linkage.to_csv(file_name)
    

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [68]:
df_predictions = linker.predict()

linker.comparison_viewer_dashboard(df_predictions, "scv.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame
IFrame(
    src="./scv.html", width="100%", height=1200
)  

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df_predictions = linker.predict(threshold_match_weight= 11)

linker.comparison_viewer_dashboard(df_predictions, "scv.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame
IFrame(
    src="./scv.html", width="100%", height=1200
)  

In [99]:
# Scenario 3
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_level_library as cll
# Define Comparisons 
tfcomparison_syn_g1_first_name = {
    "output_column_name": "syn_g1_firstname",
    "comparison_description": "G1 First name jaro winkler",
    "comparison_levels": [
        {
            "sql_condition": "syn_g1_firstname_l IS NULL OR syn_g1_firstname_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "syn_g1_firstname_l = syn_g1_firstname_r",
            "label_for_charts": "Exact match",
            "tf_adjustment_column": "syn_g1_firstname",
            "tf_adjustment_weight": 1.0,
            "tf_minimum_u_value": 0.001,
        },
        {
            "sql_condition": 'jaro_winkler_similarity("syn_g1_firstname_l", "syn_g1_firstname_r") >= 0.8',
            "label_for_charts": "Jaro_winkler_similarity 0.8",
            "tf_adjustment_column": "syn_g1_firstname",
            "tf_adjustment_weight": 0.5,
            "tf_minimum_u_value": 0.001,

        },
        {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
    ],
}

tfcomparison_syn_g0_surname = {
    "output_column_name": "syn_g0_surname",
    "comparison_description": "G0 surname jaro winkler",
    "comparison_levels": [
        {
            "sql_condition": "syn_g0_surname_l IS NULL OR syn_g0_surname_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "syn_g0_surname_l = syn_g0_surname_r",
            "label_for_charts": "Exact match",
             "tf_adjustment_column": "syn_g0_surname",
            "tf_adjustment_weight": 1.0,
            "tf_minimum_u_value": 0.001,
        },
        {
            "sql_condition": 'jaro_winkler_similarity("syn_g0_surname_l", "syn_g0_surname_r") >= 0.8',
            "label_for_charts": "Jaro_winkler_similarity 0.8",
            "tf_adjustment_column": "syn_g0_surname",
            "tf_adjustment_weight": 0.5,
            "tf_minimum_u_value": 0.001,

        },
        {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
    ],
}

tfcomparison_syn_g1_surname = {
    "output_column_name": "syn_g1_surname",
    "comparison_description": "G1 surname jaro winkler",
    "comparison_levels": [
        {
            "sql_condition": "syn_g1_surname_l IS NULL OR syn_g1_surname_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "syn_g1_surname_l = syn_g1_surname_r",
            "label_for_charts": "Exact match",
            "tf_adjustment_column": "syn_g1_surname",
            "tf_adjustment_weight": 1.0,
            "tf_minimum_u_value": 0.001,
        },
        {
            "sql_condition": 'jaro_winkler_similarity("syn_g1_surname_l", "syn_g1_surname_r") >= 0.8',
            "label_for_charts": "Jaro_winkler_similarity 0.8",     
            "tf_adjustment_column": "syn_g1_surname",
            "tf_adjustment_weight": 0.5,
            "tf_minimum_u_value": 0.001,

        },
        {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
    ],
}

comparison_date = {
    "output_column_name": "g1_dob_arc1",
    "comparison_description": "exact match vs anything else",
    "comparison_levels": [
        cll.null_level("g1_dob_arc1"),
        cll.exact_match_level("g1_dob_arc1", m_probability =  0.999),
        cll.else_level(m_probability = 0.001),
    ],
}

comparison_gender = {
    "output_column_name": "g1_gender_arc",
    "comparison_description": "exact match vs anything else",
    "comparison_levels": [
        cll.null_level("g1_gender_arc"),
        cll.exact_match_level("g1_gender_arc", m_probability =  0.999),
        cll.else_level(m_probability = 0.001),
    ],
}




In [100]:
# Scenario 3,  probabilistic linkage with term frequency adjustment
from splink.duckdb.blocking_rule_library import block_on
settings_prob_tf = {
    "link_type": "link_only",
    "probability_two_random_records_match": 1/13281,
    "blocking_rules_to_generate_predictions":[     
        block_on(["syn_g1_firstname"]),
        block_on(["syn_g1_surname"]),
        block_on(["syn_g0_surname"]),
        block_on(["g1_dob_arc1", "g1_gender_arc"]),
    ],  
    "comparisons":[
        tfcomparison_syn_g1_first_name,
        tfcomparison_syn_g1_surname,
        tfcomparison_syn_g0_surname,
        comparison_date,
        comparison_gender,      
    ],
    "retain_matching_columns":True,
    "retain_intermediate_calculation_columns":True,
    "additional_columns_to_retain": [
        "ethgroup",
        "imddecile",
        "maternal_agecat",
    ]
}

linker = DuckDBLinker(
    [gold_df, df],
    settings_dict= settings_prob_tf,
    input_table_aliases=["df_left","df_right"]
)


#linker.save_model_to_json(out_path = "linkage_outputs\\dataset2\\scenario1_dataset2\\probablistic_linkage_tf_settings.json", overwrite = True)


In [101]:
# Estimate U using random sampling
linker.estimate_u_using_random_sampling(max_pairs=2e7, seed = 1)


----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - syn_g1_firstname (no m values are trained).
    - syn_g1_surname (no m values are trained).
    - syn_g0_surname (no m values are trained).


In [102]:
# Use labelled data for m probability.
linker.estimate_m_from_label_column("unique_id")

------- Estimating m probabilities using from column unique_id -------
m probability not trained for g1_dob_arc1 - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.
m probability not trained for g1_gender_arc - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.

Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [19]:
linker.tf_adjustment_chart("syn_g1_surname")


C:\Users\uctvjla\OneDrive - University College London\Desktop\splink\splink_synthetic_data-main\venv\lib\site-packages\splink\linker.py:3343: UserWarning: Values [None] from `vals_to_include` were not found in the dataset so are not included in the chart.
  return tf_adjustment_chart(


alt.HConcatChart(...)

In [20]:
linker.tf_adjustment_chart("syn_g0_surname")


C:\Users\uctvjla\OneDrive - University College London\Desktop\splink\splink_synthetic_data-main\venv\lib\site-packages\splink\linker.py:3343: UserWarning: Values [None] from `vals_to_include` were not found in the dataset so are not included in the chart.
  return tf_adjustment_chart(


alt.HConcatChart(...)

In [21]:
linker.tf_adjustment_chart("syn_g1_firstname")


C:\Users\uctvjla\OneDrive - University College London\Desktop\splink\splink_synthetic_data-main\venv\lib\site-packages\splink\linker.py:3343: UserWarning: Values [None] from `vals_to_include` were not found in the dataset so are not included in the chart.
  return tf_adjustment_chart(


alt.HConcatChart(...)

In [22]:
linker.m_u_parameters_chart()

alt.HConcatChart(...)

In [23]:
linker.match_weights_chart()


alt.VConcatChart(...)

In [15]:
linker.match_weights_histogram(linker.predict())


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.Chart(...)

In [29]:
df = linker.predict(threshold_match_weight= -0.5)
records = df.as_record_dict(limit=10)
linker.waterfall_chart(records)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.LayerChart(...)

In [103]:
results = linker.predict()
prob_linkage = results.as_pandas_dataframe()
prob_linkage.to_csv(f"linkage_outputs\\scen0\\dataset{data_set}\\probabilistic_tf_nothreshold.csv")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
threshold_values = [-3, -2, -1, -0.5, 1, 2, 4, 5, 6]
for v in threshold_values:
    results = linker.predict(threshold_match_weight = v)
    prob_linkage = results.as_pandas_dataframe()
    file_name = f"linkage_outputs\\scen0\\dataset5\\tf_{v}.csv"
    prob_linkage.to_csv(file_name)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [81]:
df_predictions = linker.predict(threshold_match_weight= -5)

linker.comparison_viewer_dashboard(df_predictions, "scv.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame
IFrame(
    src="./scv.html", width="100%", height=1200
)  

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
linker.unlinkables_chart()

alt.LayerChart(...)

In [18]:
linker.parameter_estimate_comparisons_chart()

alt.Chart(...)